In [1]:
import warnings
warnings.filterwarnings("ignore")

In [13]:
%%capture
!pip install -qU torch==2.3.1 \
transformers==4.41.2 \
accelerate==0.31.0 \
pycaret==3.3.2 \
ipywidgets==8.1.3 \
transitions==0.9.2 \
graphviz==0.20.3

In [23]:
# Standard library imports
import copy
import traceback
from enum import Enum, auto
from pathlib import Path
from typing import Union

# Third-party library imports
import pandas as pd
import requests
import torch
import graphviz
from bs4 import BeautifulSoup
from pydantic import BaseModel, validator, HttpUrl, constr, ValidationError
from transitions import Machine, State
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from transformers.utils import logging

# Set verbosity 40+ to only display errors and critical
logging.set_verbosity_error()
# Set random seed for reproducibility
torch.random.manual_seed(0)

In [4]:
class LanguageModel:
    def __init__(self, model_name):
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            torch_dtype="auto",
            trust_remote_code=True,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        # Set to evaluation mode
        self.model.eval()

    def generate_text(self, chat_history, generation_args):
        pipe = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
        )
        with torch.no_grad():
          output = pipe(chat_history, **generation_args)
        return output[0]['generated_text'].strip()

In [5]:
class DatasetLocationModel(BaseModel):
    location: Union[HttpUrl, constr(strip_whitespace=True)]

    @validator('location', pre=True)
    def check_location(cls, v):
        if cls.is_valid_local_path(v):
            if cls.has_valid_extension(v):
                return v
            raise ValueError('The local path does not point to a CSV or Parquet file')

        if cls.is_valid_url(v):
            if cls.has_valid_extension(v):
                return v
            raise ValueError('The URL does not point to a CSV or Parquet file')

        raise ValueError('The provided location is not a valid URL or local path')

    @staticmethod
    def is_valid_url(url: str) -> bool:
        try:
            HttpUrl(url=url)
            return True
        except ValidationError:
            return False

    @staticmethod
    def is_valid_local_path(path: str) -> bool:
        return Path(path).exists()

    @staticmethod
    def has_valid_extension(path: str) -> bool:
        valid_extensions = ('.csv', '.parquet')
        return path.lower().endswith(valid_extensions)

    @classmethod
    def validate_location(cls, location: str) -> bool:
        try:
            cls(location=location)
            return True
        except ValidationError:
            return False

In [44]:
class NodeConfig:
    def __init__(self):
        # General generation arguments
        self.generation_args_template = {
            "return_full_text": False,
            "temperature": 0.0,
            "do_sample": False,
        }

        self.entity_extraction_generation_args = copy.deepcopy(self.generation_args_template)
        self.code_generation_args = copy.deepcopy(self.generation_args_template)
        self.code_fix_generation_args = copy.deepcopy(self.generation_args_template)

        self.entity_extraction_generation_args.update({"max_new_tokens": 100})
        self.code_generation_args.update({"max_new_tokens": 1000})
        self.code_fix_generation_args.update({"max_new_tokens": 600})

        # Prompts
        self.entity_extraction_prompt_template = [
          {"role": "system", "content": "You are a helpful, and accurate, AI assistant. Always follow the instructions provided by user"},
          {"role": "user", "content": None}, # Placeholder for entity to be extracted
          ]
        self.code_gen_prompt_template = [
            {"role": "system", "content": "You are a helpful, and accurate, AI assistant, that generates bug free executable python code."},
            {"role": "user", "content": "Here is the documentation on how to use the pycaret library for finding best classification model and fit it on new dataset"},
            {"role": "assistant", "content": None},  # Placeholder for library doc
            {"role": "user", "content": None},  # Placeholder for task-specific content
        ]
        self.code_fix_prompt_template = [
            {"role": "system", "content": "You are a helpful, and accurate, AI assistant, that generates bug free executable python code. Follow the output format in the example"},
            {"role": "user", "content": None},  # Placeholder for example error message and code
            {"role": "assistant", "content": None},  # Placeholder for example fixed code
            {"role": "system", "content": "You are a helpful, and accurate, AI assistant, that generates bug free executable python code."},
            {"role": "user", "content": None},  # Placeholder for actual error message and code
        ]

    def get_code_gen_prompt(self, library_doc, task, dataset_url, target_column):
        prompt = copy.deepcopy(self.code_gen_prompt_template)
        prompt[2]["content"] = library_doc
        prompt[3]["content"] = (
            f"Write code to find best model for {task} on dataset located at url: {dataset_url} "
            f"and target column:{target_column} using pycaret library, don't fit it on new data. "
            "Only generate executable code and nothing else like explanation or reasoning"
        )
        return prompt

    def get_code_fix_prompt(self, example_err_msg, example_code_with_error, example_fixed_code, errors, code):
        prompt = copy.deepcopy(self.code_fix_prompt_template)
        prompt[1]["content"] = f"Example: Only generate executable code and nothing else like explanation or reasoning. Fix this error: {example_err_msg} in the python code: {example_code_with_error}."
        prompt[2]["content"] = example_fixed_code
        prompt[4]["content"] = f"Only generate executable code and nothing else like explanation or reasoning. Fix this error: {errors} in the python code: {code}."
        return prompt


In [45]:
class Conversation:
    def __init__(self, language_model, generation_args, max_retries=5):
        self.lm = language_model
        self.generation_args = generation_args
        self.max_retries = max_retries
        self.dataset_url = None
        self.machine_learning_task = None
        self.target_column = None
        self.supported_ml_tasks = ['classification', 'regression', 'clustering']

    def extract_entities(self, user_input, entity_extraction_prompt_template):
      """Given user input, extract the dataset URL, machine learning task, and target column."""

      # Generate prompt for entity extraction
      dataset_input_prompt = copy.deepcopy(entity_extraction_prompt_template)
      machine_learning_task_input_prompt = copy.deepcopy(entity_extraction_prompt_template)
      target_column_input_prompt = copy.deepcopy(entity_extraction_prompt_template)

      dataset_input_prompt[1]["content"] = f"Given the context: {user_input}. If the context contains a url for a csv or parquet file, return the full url as response, otherwise only ouput one word False"
      machine_learning_task_input_prompt[1]["content"] = f"Given the context: {user_input}. Identify if the context mentions a machine learning task on the target column in the dataset if yes then return the machine learning task as response, like regression or classification or clustering; otherwise only ouput one word False"
      target_column_input_prompt[1]["content"] = f"Given the context: {user_input}. Identify if the context mentions a target column to be used for the machine leraning problem, if yes then return the target column  as response, otherwise only ouput one word False"

      if not self.dataset_url:
        self.dataset_url = self.lm.generate_text(dataset_input_prompt, self.generation_args)
        # Check if the URL is valid
        if not DatasetLocationModel.validate_location(self.dataset_url):
          self.dataset_url = None

      if not self.machine_learning_task:
        self.machine_learning_task = self.lm.generate_text(machine_learning_task_input_prompt, self.generation_args)
        # Check if the machine_learning_task is valid
        if not self.machine_learning_task or self.machine_learning_task.lower() not in self.supported_ml_tasks:
          self.machine_learning_task = None

      if not self.target_column:
        self.target_column = self.lm.generate_text(target_column_input_prompt, self.generation_args)
      # Check if the target_column is valid
      if self.dataset_url:
        data = None
        if self.dataset_url.endswith(".csv"):
          data = pd.read_csv(self.dataset_url, nrows= 10)
        else:
          data = pd.read_parquet(self.dataset_url).head(10)

        if not self.target_column in data.columns:
          self.target_column = None

      return None

    def is_chat_successful(self):
      return self.dataset_url and self.machine_learning_task and self.target_column


    def chat(self, entity_extraction_prompt_template):
      retries = 0
      while retries < self.max_retries and not (self.dataset_url and self.machine_learning_task and self.target_column):
        user_input = input("")
        self.extract_entities(user_input, entity_extraction_prompt_template)
        if self.dataset_url:
          print("Dataset URL:", self.dataset_url)
        else:
          print("Dataset location invalid try again")
        if self.machine_learning_task:
          print("Machine Learning Task:", self.machine_learning_task)
        else:
          print("Please choose machine task from the following: ", self.supported_ml_tasks)
        if self.target_column:
          print("Target:", self.target_column)
        else:
          print("Target columnn not found in the dataset.")

        retries += 1
        if retries == self.max_retries:
          print("Failed to extract entities after multiple retries.")

      return None

In [46]:
# Define the states for the workflow
class NodeState(Enum):
    COLLECTING_INPUTS = auto()
    GENERATING_CODE = auto()
    EXECUTING_CODE = auto()
    FIXING_ERRORS = auto()
    FINISHED = auto()
    MAX_RETRIES_REACHED = auto()

# Shared context for passing data between nodes
class WorkflowContext:
    def __init__(self, lm, documentation):
        self.inputs = None
        self.code = None
        self.fixed_code = None
        self.execution_success = None
        self.errors = None
        self.lm = lm
        self.library_doc = documentation

# Base class for all nodes in the workflow
class Node:
    def __init__(self, name, context, retries= 5):
        self.name = name
        self.context = context
        self.max_retries = retries
        self.transitions = []  # Track transitions for visualization

    def run(self):
        raise NotImplementedError("Each node must implement the run method")

    def log_transition(self, source_state, dest_state):
      self.transitions.append((source_state, dest_state))

# Node for collecting inputs
class CollectInputsNode(Node):
    def run(self):
        # Logic to collect inputs
        source_state = NodeState.COLLECTING_INPUTS
        self.context.inputs = self.collect_inputs()
        if self.inputs_collected():
            dest_state = NodeState.GENERATING_CODE
            self.log_transition(source_state, dest_state)
            return True
        else:
          dest_state = NodeState.COLLECTING_INPUTS
          self.log_transition(source_state, dest_state)
          return False

    def collect_inputs(self):
        # Define generation arguments
        config = NodeConfig()
        # Initialize Conversation
        conversor = Conversation(self.context.lm, config.entity_extraction_generation_args, self.max_retries)
        conversor.chat(config.entity_extraction_prompt_template)
        if conversor.is_chat_successful():
          return {'dataset_url': conversor.dataset_url, 'machine_learning_task': conversor.machine_learning_task, 'target_column': conversor.target_column}
        return None


    def inputs_collected(self):
        # Implement logic to check if inputs are collected
        return self.context.inputs is not None

# Node for generating code
class GenerateCodeNode(Node):
    def run(self):
        # Logic to generate code

        config = NodeConfig()
        prompt = config.get_code_gen_prompt(
            self.context.library_doc,
            self.context.inputs['machine_learning_task'],
            self.context.inputs['dataset_url'],
            self.context.inputs['target_column']
        )
        self.context.code = self.generate_code(prompt, config.code_generation_args)
        # self.context.code = " " + self.context.code
        source_state = NodeState.GENERATING_CODE
        dest_state = NodeState.EXECUTING_CODE
        self.log_transition(source_state, dest_state)
        return True

    def generate_code(self, code_gen_prompt, code_generation_args):
        # Implement code generation logic
        return self.context.lm.generate_text(code_gen_prompt, code_generation_args)

# Node for executing code
class ExecuteCodeNode(Node):
    def run(self):
        # Logic to execute code
        source_state = NodeState.EXECUTING_CODE
        success, errors = self.execute_code(self.context.code)
        self.context.execution_success = success
        self.context.errors = errors
        if success:
            dest_state = NodeState.FINISHED
        else:
            dest_state = NodeState.FIXING_ERRORS
        self.log_transition(source_state, dest_state)

        return success

    def execute_code(self, code):
        # code execution logic
        try:
            print(code)
            exec(code)
            return True, None  # Indicate successful execution
        except Exception as e:
            error_message = str(traceback.format_exc())
            return False, [error_message]

# Node for fixing errors
class FixErrorsNode(Node):
    def __init__(self, name, context, max_retries=3):
        super().__init__(name, context)
        self.retries = 0
        self.max_retries = max_retries

    def run(self):
        # Logic to fix errors
        source_state = NodeState.FIXING_ERRORS
        self.context.fixed_code = self.fix_errors(self.context.errors, self.context.code)
        self.context.code = self.context.fixed_code
        self.retries += 1
        if self.retries >= self.max_retries:
            dest_state = NodeState.MAX_RETRIES_REACHED
        else:
            dest_state = NodeState.EXECUTING_CODE
        self.log_transition(source_state, dest_state)
        return self.retries < self.max_retries

    def fix_errors(self, errors, code):
        # Implement error fixing logic
        example_err_msg = (
            "Traceback (most recent call last):\n"
            '  File "<ipython-input-48-a50281d5d318>", line 3, in <cell line: 2>\n'
            "    exec(code_2_run)\n"
            '  File "<string>", line 6, in <module>\n'
            "NameError: name 'adde_two_numbers' is not defined"
        )

        example_code_with_error = (
            "def add_two_numbers(a, b):\n"
            "    return a + b\n\n"
            "a = 10\n"
            "b = 5\n"
            "print(adde_two_numbers(a, b))"
        )

        example_fixed_code = (
            "# Here is the fixed code\n"
            "def add_two_numbers(a, b):\n"
            "    return a + b\n\n"
            "a = 10\n"
            "b = 5\n"
            "print(add_two_numbers(a, b))"
        )
        prompt = NodeConfig().get_code_fix_prompt(
            example_err_msg,
            example_code_with_error,
            example_fixed_code,
            errors,
            code
        )
        return self.context.lm.generate_text(prompt, NodeConfig().code_fix_generation_args)

In [47]:
# The workflow graph using the transitions library
class Workflow:
    states = [
        'collecting_inputs',
        'generating_code',
        'executing_code',
        'fixing_errors',
        'finished',
        'max_retries_reached'
    ]

    def __init__(self, lm, documentation):
        self.context = WorkflowContext(lm, documentation)
        self.nodes = {
            'collecting_inputs': CollectInputsNode('collect_inputs', self.context),
            'generating_code': GenerateCodeNode('generate_code', self.context),
            'executing_code': ExecuteCodeNode('execute_code', self.context),
            'fixing_errors': FixErrorsNode('fix_errors', self.context),
        }
        self.current_node = self.nodes['collecting_inputs']
        self.state = NodeState.COLLECTING_INPUTS
        self.transitions = []  # Track global transitions

        # Set up the state machine
        self.machine = Machine(model=self, states=Workflow.states, initial='collecting_inputs')

        # Define transitions between states
        self.machine.add_transition('collect_inputs', 'collecting_inputs', 'generating_code', conditions='run_collecting_inputs')
        self.machine.add_transition('generate_code', 'generating_code', 'executing_code', conditions='run_generating_code')
        self.machine.add_transition('execute_code', 'executing_code', 'finished', conditions='run_executing_code')
        self.machine.add_transition('execution_failed', 'executing_code', 'fixing_errors')
        self.machine.add_transition('fix_errors', 'fixing_errors', 'executing_code', conditions='run_fixing_errors')
        self.machine.add_transition('max_retries', '*', 'max_retries_reached')

    def run_collecting_inputs(self):
        return self.nodes['collecting_inputs'].run()

    def run_generating_code(self):
        return self.nodes['generating_code'].run()

    def run_executing_code(self):
        success = self.nodes['executing_code'].run()
        if success:
            return True
        else:
            self.execution_failed()
            return False

    def run_fixing_errors(self):
        can_retry = self.nodes['fixing_errors'].run()
        if can_retry:
            return True
        else:
            self.max_retries()
            return False

    def run(self):
        while self.state not in ['finished', 'max_retries_reached']:
            print('Inside Workflow, current state is:', self.state)
            if self.state == 'collecting_inputs':
                self.collect_inputs()
            elif self.state == 'generating_code':
                self.generate_code()
            elif self.state == 'executing_code':
                self.execute_code()
            elif self.state == 'fixing_errors':
                self.fix_errors()

            # Collect global transitions for the workflow graph
            self.transitions.append((self.state, self.current_node.name, self.state))

    def visualize_workflow(self, filename='workflow_graph'):
        dot = graphviz.Digraph(comment='Workflow Execution')
        # Add nodes and edges to the graph
        for node_name, node in self.nodes.items():
            dot.node(node_name, node_name)
            for (source_state, dest_state) in node.transitions:
                dot.edge(source_state.name, dest_state.name, label=f'{node_name}')

        # Render the graph to a file
        dot.render(filename, format='png', cleanup=True)
        print(f"Workflow graph saved as {filename}.png")

In [12]:
model_name = "microsoft/Phi-3-mini-128k-instruct"
lm = LanguageModel(model_name)

config.json:   0%|          | 0.00/3.48k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [48]:
def fetch_raw_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return None

def parse_html(html_content):
    soup = BeautifulSoup(html_content, 'lxml')

    # Initialize a list to keep all elements in order
    elements = []

    # Extract all relevant tags
    for tag in soup.find_all(['p', 'code', 'div'], recursive=True):
        if tag.name == 'p':
            elements.append(('paragraph', tag.get_text()))
        if tag.name == 'code':
            elements.append(('code', tag.get_text()))
        elif tag.name == 'div' and 'section' in tag.get('class', []):
            elements.append(('section', tag.get_text()))

    extracted_content = ""
    for i, (element_type, text) in enumerate(elements):
        extracted_content = extracted_content + "\n" + f"{element_type.capitalize()} {i + 1}: {text}"

    return extracted_content

# Example URL
#url = "https://pycaret.gitbook.io/docs/get-started/quickstart"
url = "https://raw.githubusercontent.com/abhimanyu729/GenAIPlayground/main/aisc_demo/pycaret_documentation.html"

# Fetch and parse HTML content
raw_html = fetch_raw_html(url)
documentation_context = None
if raw_html:
    documentation_context = parse_html(raw_html)

In [49]:
workflow = Workflow(lm, documentation_context)

In [50]:
workflow.run()
## Test Cases:
# 1. The dataset I want to use is Titanic, and the column to classify on is Survived
# 2. I don't know where the data is
# 3. You can find data here: https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
# 4. I'd like you to apply regression on the target column
# 5. Machine Learning Task: Clustering

Inside Workflow, current state is: collecting_inputs
The dataset I want to use is Titanic, and the column to classify on is Survived
Dataset location invalid try again
Machine Learning Task: Classification
Target: Survived
You can find data here: https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Dataset URL: https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv
Machine Learning Task: Classification
Target: Survived
Inside Workflow, current state is: generating_code
Inside Workflow, current state is: executing_code
 import pandas as pd
from pycaret.classification import *

# Load the dataset
data = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

# Set up the PyCaret environment
clf = setup(data=data, target='Survived')

# Train and compare models
best_model = compare_models()

# Tune the best model
tuned_model = tune_model(best_model)

# Evaluate the tuned model
evaluate_model(tuned_model)


,Description,Value
0,Session id,1482
1,Target,Survived
2,Target type,Binary
3,Original data shape,"(891, 12)"
4,Transformed data shape,"(891, 14)"
5,Transformed train set shape,"(623, 14)"
6,Transformed test set shape,"(268, 14)"
7,Numeric features,6
8,Categorical features,5
9,Rows with missing values,79.5%


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.8073,0.8425,0.6527,0.8084,0.7199,0.5763,0.5855,0.3280
et,Extra Trees Classifier,0.7207,0.7616,0.3560,0.8116,0.4862,0.3378,0.3936,0.2290
ridge,Ridge Classifier,0.7176,0.8295,0.3728,0.7926,0.4977,0.3371,0.3873,0.1080
nb,Naive Bayes,0.6373,0.7673,0.0962,0.7667,0.1665,0.0835,0.1612,0.1040
lda,Linear Discriminant Analysis,0.6228,0.5189,0.0348,0.0667,0.0457,0.0272,0.0300,0.1040
dt,Decision Tree Classifier,0.6164,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1110
rf,Random Forest Classifier,0.6164,0.8057,0.0000,0.0000,0.0000,0.0000,0.0000,0.2550
ada,Ada Boost Classifier,0.6164,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1760
gbc,Gradient Boosting Classifier,0.6164,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2700
xgboost,Extreme Gradient Boosting,0.6164,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1220


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8571,0.8729,0.6667,0.9412,0.7805,0.6791,0.7013
1,0.8730,0.8974,0.7917,0.8636,0.8261,0.7264,0.7281
2,0.8254,0.8141,0.6250,0.8824,0.7317,0.6078,0.6277
3,0.7581,0.8105,0.6522,0.6818,0.6667,0.4769,0.4772
4,0.7581,0.8202,0.5417,0.7647,0.6341,0.4612,0.4765
5,0.8226,0.7862,0.7083,0.8095,0.7556,0.6173,0.6207
6,0.8065,0.8333,0.7083,0.7727,0.7391,0.5857,0.5871
7,0.8065,0.9156,0.6250,0.8333,0.7143,0.5724,0.5860
8,0.8871,0.8717,0.7917,0.9048,0.8444,0.7565,0.7606


Fitting 10 folds for each of 10 candidates, totalling 100 fits


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [51]:
workflow.visualize_workflow(filename='workflow_graph_execution_2')

Workflow graph saved as workflow_graph_execution_2.png


In [86]:
# bug_understanding_prompt = [
#     {"role": "system", "content": "You are a helpful, and accurate, AI assistant, that's an expert in python programming"},
#     {"role": "user", "content": f"Given the following python code: {code} and the error: {errors}. Think and provide clear explanation in natural language about what's the root cause of the error. The explantion should be short and concise."},
# ]
# bug_understanding_generation_args = {
#     "max_new_tokens": 200,
#     "return_full_text": False,
#     "temperature": 0.0,
#     "do_sample": False,
# }

In [40]:
workflow.state

'finished'

In [ ]:
# Why do you think the error exists provide resoning
# use reasoning error and
# input code to fix it.